<a href="https://colab.research.google.com/github/DarthCoder501/GAAP/blob/main/Baseline_Impressions_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow transformers scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.metrics import roc_auc_score, classification_report, multilabel_confusion_matrix
import warnings
warnings.filterwarnings('ignore')

Preprocessing text data...
Train samples after cleaning: 4460
Test samples after cleaning: 1116
Loading Bio_ClinicalBERT...


Some layers from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning 

Tokenizing text...


In [ ]:
# Set random seeds for reproducibility
RANDOM_SEED = 42

# Python random seed
random.seed(RANDOM_SEED)

# NumPy random seed
np.random.seed(RANDOM_SEED)

# TensorFlow random seed
tf.random.set_seed(RANDOM_SEED)

In [ ]:
# Load data
train = pd.read_csv("/content/Train Chronological (1).csv", parse_dates=["note_DATETIME"])
test = pd.read_csv("/content/Impressions Chronological Data (1).csv", parse_dates=["note_DATETIME"])

In [ ]:
# Abbreviations dictionary
abbreviations = {
    "MM": "millimeter",
    "CT": "computed tomography",
    "HCW": "healthcare Worker",
    "BAC": "bronchioloalveolar carcinoma",
    "ED": "emergency department",
    "PT": "pacific time",
    "MRI": "magnetic resonance imaging",
    "RN": "registered nurse",
    "PACS": "picture archiving and communication system",
    "PET": "positron emission tomography",
    "SVC": "superior vena cava",
    "CM": "centimeter",
    "RLL": "right lower lobe",
    "RUL": "right upper lobe",
    "LAD": "left anterior descending artery",
    "TB": "tuberculosis",
    "IPMT": "intraductal papillary mucinous tumor",
    "IVC": "inferior vena cava",
    "PE": "pulmonary embolism",
    "PEs": "pulmonary embolisms",
    "FDG": "fluorodeoxyglucose",
    "SFV": "superficial femoral vein",
    "DVT": "deep vein thrombosis",
    "SMA": "superior mesenteric artery",
    "NSIP": "nonspecific interstitial pneumonia",
    "SITU": "in its original place",
    "HR": "hour",
    "4A": "the superior part of the left medial segment of the liver",
    "PST": "pacific standard time",
    "ID": "identification",
    "CTA": "computed tomography angiography",
    "NG": "nasogastric",
    "IPMN": "intraductal papillary mucinous neoplasm",
    "UIP": "usual interstitial pneumonia",
    "ER": "emergency room",
    "ARDS": "acute respiratory distress syndrome",
    "MRN": "medical record number",
    "RV": "right ventricular",
    "CHF": "congestive heart failure",
    "PEG": "percutaneous endoscopic gastrostomy",
    "PICC": "peripherally inserted central catheter",
    "GI": "gastrointestinal",
    "ASD": "atrial septal defect",
    "MR": "mitral regurgitation",
    "EST": "eastern standard time",
    "CTs": "computed tomographies",
    "3D": "three dimensional",
    "MAC": "mycobacterium avium complex",
    "MICU": "medical intensive care unit",
    "MAI": "mycobacterium avium-intracellulare",
    "PJP": "pneumocystis jirovecii pneumonia",
    "LIMA": "left internal mammary artery",
    "LV": "left ventricle",
    "EGD": "esophagogastroduodenoscopy",
    "PAU": "penetrating atherosclerotic ulcer",
    "VP": "ventriculoperitoneal",
    "CSF": "cerebrospinal fluid",
    "HCC": "hepatocellular carcinoma",
    "SABR": "stereotactic ablative radiotherapy",
    "ILD": "interstitial lung disease",
    "IVP": "intravenous pyelogram",
    "MRCP": "magnetic resonance cholangiopancreatography",
    "IV": "intravenous",
    "RCA": "right coronary artery",
    "COVID": "coronavirus disease",
    "2D": "two dimensional",
    "SMV": "superior mesenteric vein",
    "FNA": "fine needle aspiration",
    "BAL": "bronchoalveolar Lavage",
    "AVMs": "arteriovenous malformations",
    "AVM": "arteriovenous malformation",
    "MRA": "magnetic resonance angiography",
    "AP": "anteroposterior",
    "MRIs": "magnetic resonance imaging",
    "COVID19": "coronavirus disease 2019",
    "BHD": "birt-hogg-dube",
    "CTEPH": "chronic thromboembolic pulmonary hypertension",
    "RML": "right middle lobe",
    "NGT": "nasogastric tube",
    "GE": "gastroesophageal",
    "MDS": "myelodysplastic syndrome",
    "UVJ": "ureterovesical junction",
    "ERCP": "endoscopic retrograde cholangiopancreatography",
    "OP": "organizing pneumonia",
    "IJ": "internal jugular",
    "VSD": "ventricular septal defect",
    "EMR": "electronic medical record",
    "TE": "tracheoesophageal",
    "AV": "arteriovenous",
    "PAN": "polyarteritis nodosa",
    "III": "third",
    "SLE": "systemic lupus erythematosus",
    "CTS": "computed tomographies",
    "IPF": "idiopathic pulmonary fibrosis",
    "3MM": "three millimeters",
    "4MM": "four millimeters",
    "PAPVR": "partial anomalous pulmonary venous return",
    "ANCA": "antineutrophil cytoplasmic antibodies",
    "VQ": "ventilation-perfusion",
    "PA": "pulmonary artery",
    "PCP": "pneumocystis pneumonia",
    "CMV": "cytomegalovirus",
    "RVH": "right ventricular hypertrophy",
    "TSH": "thyroid stimulating hormone",
    "CBD": "common bile duct",
    "BNP": "brain natriuretic peptide",
    "16MM": "sixteen millimeters",
    "NP": "nurse practitioner",
    "CVC": "central venous catheter",
    "SVG": "saphenous vein graft",
    "PDA": "posterior descending artery",
    "VIII": "eighth",
    "ICU": "intensive care unit",
    "CPR": "cardiopulmonary resuscitation",
    "DAH": "diffuse alveolar hemorrhage",
    "PAP": "pulmonary alveolar proteinosis",
    "II": "second",
    "ENT": "ear, nose, and throat",
    "FNH": "focal nodular hyperplasia",
    "LLL": "left lower lobe",
    "CTPA": "computed tomography pulmonary angiography",
    "LA": "left atrium",
    "ABPA": "allergic bronchopulmonary aspergillosis",
    "IMA": "inferior mesenteric artery",
    "RT": "right",
    "CCU": "coronary care unit",
    "ALS": "amyotrophic lateral sclerosis",
    "LT": "left",
    "RCC": "renal cell carcinoma",
    "AML": "angiomyolipoma",
    "HCG": "human chorionic gonadotropin",
    "IJV": "internal jugular vein",
    "LE": "lower extremity",
    "ASAP": "as soon as possible",
    "1L": "one liter",
    "IHSS": "Idiopathic hypertrophic subaortic stenosis",
    "13MM": "thirteen millimeters",
    "PFO": "patent foramen ovale",
    "CCA": "common carotid artery",
    "SCA": "subclavian artery",
    "ANS": "anteromedial basal subsegmental artery",
    "IgG4": "Immunoglobulin G4",
    "ICD": "implantable cardioverter-defibrillator",
    "T9": "ninth thoracic vertebrae",
    "CVICU": "cardiovascular intensive care unit",
    "T12": "twelfth thoracic vertebra",
    "L5": "fifth lumbar vertebra",
    "L1": "first lumbar vertebra",
    "L3": "third lumbar vertebra",
    "T4": "fourth thoracic vertebrae",
    "T5": "fifth thoracic vertebra",
    "T7": "seventh thoracic vertebra",
    "T8": "eighth thoracic vertebra",
    "T10": "tenth thoracic vertebra",
    "L2": "second lumbar vertebra",
    "8MM": "eight millimeters",
    "T2": "second thoracic vertebra",
    "IUD": "intrauterine device",
    "T3": "third thoracic vertebrae",
    "T6": "sixth thoracic vertebrae",
    "C7": "seventh cervical vertebra",
    "S4": "fourth heart sound",
    "T11": "eleventh thoracic vertebra",
    "L4": "fourth lumbar vertebra",
    "T1": "first thoracic vertebra",
    "S1": "first heart sound",
    "PAH": "pulmonary arterial hypertension",
    "S9": "ninth heart sound",
    "IMH": "intramural hematoma",
    "VATS": "video-assisted thoracoscopic surgery",
    "S2": "second heart sound",
    "LVAD": "left ventricular assist device",
}

In [ ]:
# Data preprocessing
print("Preprocessing text data...")
train["impressions_clean"] = train["impressions"].replace(abbreviations, regex=True)
test["impressions_clean"] = test["impressions"].replace(abbreviations, regex=True)

In [ ]:
# Clean text
train["impressions_clean"] = train["impressions_clean"].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)
test["impressions_clean"] = test["impressions_clean"].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)

In [ ]:
# Handle missing values
train = train.dropna(subset=["impressions_clean"])
test = test.dropna(subset=["impressions_clean"])

In [ ]:
# Remove empty strings after cleaning
train = train[train["impressions_clean"].str.strip() != ""]
test = test[test["impressions_clean"].str.strip() != ""]

print(f"Train samples after cleaning: {len(train)}")
print(f"Test samples after cleaning: {len(test)}")

In [ ]:
# Load tokenizer and model
print("Loading Bio_ClinicalBERT...")
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
bert_model = TFAutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
# Tokenize text
max_seq_length = 128
print("Tokenizing text...")
train_encodings = tokenizer(
    train["impressions_clean"].tolist(),
    padding=True,
    truncation=True,
    max_length=max_seq_length,
    return_tensors="tf"
)
test_encodings = tokenizer(
    test["impressions_clean"].tolist(),
    padding=True,
    truncation=True,
    max_length=max_seq_length,
    return_tensors="tf"
)

In [ ]:
# Extract embeddings function (fixed)
def extract_embeddings_in_batches(model, encodings, batch_size=32):
    """Extract CLS token embeddings in batches to manage memory"""
    all_embeddings = []
    num_samples = encodings.input_ids.shape[0]

    for i in range(0, num_samples, batch_size):
        end_idx = min(i + batch_size, num_samples)

        batch_input_ids = encodings.input_ids[i:end_idx]
        batch_attention_mask = encodings.attention_mask[i:end_idx]

        # Get embeddings for this batch
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        # Use CLS token (first token) embeddings
        batch_embeddings = outputs.last_hidden_state[:, 0, :]
        all_embeddings.append(batch_embeddings.numpy())

        if (i // batch_size + 1) % 10 == 0:
            print(f"Processed {i + batch_size} / {num_samples} samples")

    return np.concatenate(all_embeddings, axis=0)

Extracting embeddings...
Processed 320 / 4460 samples
Processed 640 / 4460 samples
Processed 960 / 4460 samples
Processed 1280 / 4460 samples
Processed 1600 / 4460 samples
Processed 1920 / 4460 samples
Processed 2240 / 4460 samples
Processed 2560 / 4460 samples
Processed 2880 / 4460 samples
Processed 3200 / 4460 samples
Processed 3520 / 4460 samples
Processed 3840 / 4460 samples
Processed 4160 / 4460 samples
Processed 4480 / 4460 samples
Processed 320 / 1116 samples
Processed 640 / 1116 samples
Processed 960 / 1116 samples
Shape of X_train: (4460, 768)
Shape of X_test: (1116, 768)


In [ ]:
# Extract embeddings
print("Extracting embeddings...")
batch_size = 32  # Reduced batch size for stability
X_train = extract_embeddings_in_batches(bert_model, train_encodings, batch_size)
X_test = extract_embeddings_in_batches(bert_model, test_encodings, batch_size)

In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

In [ ]:
# Define targets
targets = ["1_month_readmission", "6_month_readmission", "12_month_readmission", "pe_positive"]

# Check if target columns exist and handle missing values
print("Preparing target variables...")
for target in targets:
    if target not in train.columns:
        print(f"Warning: {target} not found in train data")
    if target not in test.columns:
        print(f"Warning: {target} not found in test data")

# Create target arrays & handle missing columns
available_targets = [t for t in targets if t in train.columns and t in test.columns]
print(f"Available targets: {available_targets}")

if not available_targets:
    print("No target columns found! Please check column names.")
    # Print available columns for debugging
    print("Train columns:", train.columns.tolist())
    print("Test columns:", test.columns.tolist())
else:
    y_train = train[available_targets].fillna(0).astype(int)
    y_test = test[available_targets].fillna(0).astype(int)

    print("Target distribution in training data:")
    for target in available_targets:
        pos_count = y_train[target].sum()
        total_count = len(y_train)
        print(f"{target}: {pos_count}/{total_count} ({pos_count/total_count*100:.1f}% positive)")

Preparing target variables...
Available targets: ['1_month_readmission', '6_month_readmission', '12_month_readmission', 'pe_positive']
Target distribution in training data:
1_month_readmission: 201/4460 (4.5% positive)
6_month_readmission: 593/4460 (13.3% positive)
12_month_readmission: 813/4460 (18.2% positive)
pe_positive: 1096/4460 (24.6% positive)


In [ ]:
# Build the multi-label classification model
print("Building model...")
input_layer = Input(shape=(X_train.shape[1],), name='embeddings_input')
dense1 = Dense(256, activation="relu", name='dense1')(input_layer)
dropout1 = Dropout(0.3, name='dropout1', seed=RANDOM_SEED)(dense1)
dense2 = Dense(128, activation="relu", name='dense2')(dropout1)
dropout2 = Dropout(0.2, name='dropout2', seed=RANDOM_SEED)(dense2)

# Create task-specific output heads
outputs = []
for target in available_targets:
    output = Dense(1, activation="sigmoid", name=target)(dropout2)
    outputs.append(output)

# Create and compile model
classification_model = Model(inputs=input_layer, outputs=outputs)

# For multi-output models, we need to provide metrics for each output
if len(available_targets) == 1:
    # Single output - simple metrics list
    classification_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
else:
    # Multiple outputs - provide metrics list matching number of outputs
    # Each entry in the list corresponds to one output
    metrics_list = [['accuracy'] for _ in range(len(available_targets))]
    classification_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=metrics_list
    )

Building model...
Model architecture:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ embeddings_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense1 (Dense)      │ (None, 256)       │    196,864 │ embeddings_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout1 (Dropout)  │ (None, 256)       │          0 │ dense1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense2 (Dense)      │ (None, 128)       │     32,896 │ dropout1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout2 (Dropout)  │ (None, 128)       │          0 │ dense2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 1_month_readmission │ (None, 1)         │        129 │ dropout2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 6_month_readmission │ (None, 1)         │        129 │ dropout2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 12_month_readmissi… │ (None, 1)         │        129 │ dropout2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pe_positive (Dense) │ (None, 1)         │        129 │ dropout2[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 230,276 (899.52 KB)

 Trainable params: 230,276 (899.52 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("Model architecture:")
classification_model.summary()

In [ ]:
# Training
print("Training model...")
history = classification_model.fit(
    X_train,
    [y_train[target].values for target in available_targets],
    validation_data=(X_test, [y_test[target].values for target in available_targets]),
    epochs=10,
    batch_size=32,
    verbose=1
)

Training model...
Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - 12_month_readmission_accuracy: 0.8009 - 12_month_readmission_loss: 0.5106 - 1_month_readmission_accuracy: 0.9327 - 1_month_readmission_loss: 0.2374 - 6_month_readmission_accuracy: 0.8421 - 6_month_readmission_loss: 0.4298 - loss: 1.6644 - pe_positive_accuracy: 0.7737 - pe_positive_loss: 0.4867 - val_12_month_readmission_accuracy: 0.7679 - val_12_month_readmission_loss: 0.5505 - val_1_month_readmission_accuracy: 0.9194 - val_1_month_readmission_loss: 0.2866 - val_6_month_readmission_accuracy: 0.8172 - val_6_month_readmission_loss: 0.4857 - val_loss: 1.6389 - val_pe_positive_accuracy: 0.8584 - val_pe_positive_loss: 0.3186
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - 12_month_readmission_accuracy: 0.8297 - 12_month_readmission_loss: 0.4717 - 1_month_readmission_accuracy: 0.9570 - 1_month_readmission_loss: 0.1810 - 6_month_readmission_accuracy: 0.8758 - 6_month_readmission_loss: 0.3848 - loss: 1.4269 - pe_po

In [ ]:
# Evaluation
print("\nEvaluating model...")
predictions = classification_model.predict(X_test, batch_size=32)

# Ensure predictions is a list for multi-output models
if len(available_targets) == 1:
    predictions = [predictions]

# Calculate metrics for each target
results = {}
for i, target in enumerate(available_targets):
    y_true = y_test[target].values
    y_pred = predictions[i].ravel()
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate AUC-ROC (handle edge case where all labels are the same class)
    try:
        if len(np.unique(y_true)) > 1:
            auc_score = roc_auc_score(y_true, y_pred)
        else:
            auc_score = np.nan
            print(f"Warning: {target} has only one class in test set, AUC cannot be calculated")
    except Exception as e:
        auc_score = np.nan
        print(f"Error calculating AUC for {target}: {e}")

    results[target] = {
        'auc_roc': auc_score,
        'accuracy': np.mean(y_true == y_pred_binary),
        'precision': np.sum((y_pred_binary == 1) & (y_true == 1)) / max(np.sum(y_pred_binary == 1), 1),
        'recall': np.sum((y_pred_binary == 1) & (y_true == 1)) / max(np.sum(y_true == 1), 1)
    }

    print(f"\n{target} Results:")
    print(f"  AUC-ROC: {auc_score:.4f}" if not np.isnan(auc_score) else "  AUC-ROC: N/A")
    print(f"  Accuracy: {results[target]['accuracy']:.4f}")
    print(f"  Precision: {results[target]['precision']:.4f}")
    print(f"  Recall: {results[target]['recall']:.4f}")

    # Classification report
    print(f"\nClassification Report for {target}:")
    print(classification_report(y_true, y_pred_binary, zero_division=0))


Evaluating model...
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

1_month_readmission Results:
  AUC-ROC: 0.5616
  Accuracy: 0.9194
  Precision: 0.0000
  Recall: 0.0000

Classification Report for 1_month_readmission:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1026
           1       0.00      0.00      0.00        90

    accuracy                           0.92      1116
   macro avg       0.46      0.50      0.48      1116
weighted avg       0.85      0.92      0.88      1116


6_month_readmission Results:
  AUC-ROC: 0.5229
  Accuracy: 0.8172
  Precision: 0.0000
  Recall: 0.0000

Classification Report for 6_month_readmission:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       912
           1       0.00      0.00      0.00       204

    accuracy                           0.82      1116
   macro avg       0.41      0.50      0.45      1116
weighted avg       0.67      0.82    

In [ ]:
# Overall model performance
print("\n" + "="*50)
print("OVERALL MODEL PERFORMANCE SUMMARY")
print("="*50)

avg_auc = np.nanmean([results[target]['auc_roc'] for target in available_targets])
avg_accuracy = np.mean([results[target]['accuracy'] for target in available_targets])
avg_precision = np.mean([results[target]['precision'] for target in available_targets])
avg_recall = np.mean([results[target]['recall'] for target in available_targets])

print(f"Average AUC-ROC: {avg_auc:.4f}")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")

print("\nModel training completed successfully!")


OVERALL MODEL PERFORMANCE SUMMARY
Average AUC-ROC: 0.6415
Average Accuracy: 0.8526
Average Precision: 0.2323
Average Recall: 0.1449

Model training completed successfully!
